In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)

In [2]:
LEGIS_TEXT = """\
Sygnatura akt II AKa 335/15

WYROK
W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ
Dnia 20 stycznia 2016 r.

Sąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:

Przewodniczący: SSA Stanisław Rączkowski (spr.)

Sędziowie: SSA Edward Stelmasik

SSO del. do SA Edyta Gajgał

Protokolant: Beata Sienica

przy udziale prokuratora Prokuratury Apelacyjnej Leszka Pruskiego

po rozpoznaniu w dniu 20 stycznia 2016 r.

sprawy G. K.

oskarżonego z art. 280 § 2 kk

na skutek apelacji wniesionej przez oskarżonego

od wyroku Sądu Okręgowego w Jeleniej Górze

z dnia 20 października 2015 r. sygn. akt III K 43/15

I . zmienia zaskarżony wyrok wobec oskarżonego G. K. w ten sposób, że wymierzoną temu oskarżonemu karę obniża do 3 ( trzech) lat pozbawienia wolności,

II. utrzymuje w mocy w pozostałym zakresie zaskarżony wyrok,

III. zasądza od Skarbu Państwa na rzecz adw. W. B. 600 złotych tytułem kosztów nieopłaconej pomocy prawnej udzielonej oskarżonemu G. K. z urzędu przed sądem odwoławczym oraz 138 zł tytułem zwrotu VAT,

IV. zwalnia oskarżonego od kosztów sądowych za postępowanie odwoławcze.

E. S.

S. R.

E. G.
"""

In [3]:
INSTRUCTION = '''\
You are a question-answer generator. Your goal is to generate question-answer pairs given the Context.
Do not tranlate the Context, generate questions and answers in original language.

Step 1: Identify spans that are likely to be answers to questions, identify as many as possible.
Step 2: For each identified span, generate a question.
Step 3: Respond to the question in only a few tokens concisely.
Step 4: Output in JSON format following [...]

Ensure that you distinctly label and delineate Steps 1, 2, 3, and 4.

Context: {context}

Example output: [{{"question": "generated question", "answer": "gathered answer"}}, ...]

{format_instructions}

```json
'''

In [4]:
# Define your desired data structure.
class SyntheticLegisQARecord(BaseModel):
    question: str = Field(description="Question parsed from last step of answer")
    answer: str = Field(description="Answer parsed from last step of answer")

In [5]:
output_parser = JsonOutputParser(pydantic_object=SyntheticLegisQARecord)

In [6]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"question": {"title": "Question", "description": "Question parsed from last step of answer", "type": "string"}, "answer": {"title": "Answer", "description": "Answer parsed from last step of answer", "type": "string"}}, "required": ["question", "answer"]}
```


In [7]:
prompt = ChatPromptTemplate.from_template(
    template=INSTRUCTION,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [8]:
model = ChatOpenAI(model="gpt-4")

chain = prompt | model | output_parser

chain.invoke({"context": LEGIS_TEXT})

[{'question': 'Kiedy został wydany wyrok?', 'answer': '20 stycznia 2016 r.'},
 {'question': 'Który sąd wydał wyrok?',
  'answer': 'Sąd Apelacyjny we Wrocławiu II Wydział Karny'},
 {'question': 'Kto był przewodniczącym sądu?',
  'answer': 'SSA Stanisław Rączkowski'},
 {'question': 'Kto był sędzią w sprawie?', 'answer': 'SSA Edward Stelmasik'},
 {'question': 'Kto był delegatem do Sądu Apelacyjnego?',
  'answer': 'SSA Edyta Gajgał'},
 {'question': 'Kto był protokolantem?', 'answer': 'Beata Sienica'},
 {'question': 'Kiedy sprawa została rozpoznana?',
  'answer': '20 stycznia 2016 r.'},
 {'question': 'Kto był oskarżony?', 'answer': 'G. K.'},
 {'question': 'Z jakiego artykułu oskarżony został oskarżony?',
  'answer': 'art. 280 § 2 kk'},
 {'question': 'Kiedy został wydany wyrok Sądu Okręgowego?',
  'answer': '20 października 2015 r.'},
 {'question': 'Na jaką karę został skazany oskarżony?',
  'answer': '3 lata pozbawienia wolności'},
 {'question': 'Jaka była wysokość kosztów nieopłaconej pomo

Trace: https://smith.langchain.com/public/80d26eb5-6a3e-4d7e-bcbc-5221edb7f1ce/r